###  Outubro, 8, 2024


## TP2 - Grupo 20

Afonso Martins Campos Fernandes - A102940

Luís Filipe Pinheiro Silva - A105530

### Exercicio 2 - Multiplicação de Inteiros

Considere o problema descrito no documento "+Lógica Computacional: Multiplicação de Inteiros" . Nesse documento usa-se um “Control Flow Automaton” como  modelo do programa imperativo que calcula a multiplicação de  inteiros positivos representados por vetores de bits.

In [34]:
from pysmt.shortcuts import *
from z3 import *


Pretende-se:

a. Construir um SFOTS, usando BitVec’s de tamanho $n$ , que descreva o comportamento deste autómato; para isso identifique e codifique em `Z3`  ou `pySMT`, as variáveis do modelo, o estado inicial , a relação de transição e o estado de erro.


In [35]:
def states(i,n):
    state = {}
    
    state['c'] = Int('c'+str(i))
    state['x'] = BitVec('x' + str(i),n)
    state['y'] = BitVec('y' + str(i),n)
    state['z'] = BitVec('z' + str(i),n)

    return state

def init(state,a,b):

    return And(state['c'] == 0, state['x'] == a, state['y'] == b, state['z'] == 0)

def trans(atual,prox,n):

    #transicões entre estados Ex: t0_1 -> transição entre o estado 0 e o estado 1

    valores_constantes = And(prox['x'] == atual['x'],prox['y'] == atual['y'],prox['z'] == atual['z'])

    t0_1 = And(atual['c'] == 0, prox['c'] == 1, valores_constantes)

    t1_2 = And(atual['c'] == 1, prox['c'] == 2, Not(atual['y'] == 0), URem(atual['y'],2) == 1, valores_constantes)

    t1_3 = And(atual['c'] == 1, prox['c'] == 3, Not(atual['y'] == 0), URem(atual['y'],2) == 0, valores_constantes)

    t3_1 = And(atual['c'] == 3, prox['x'] == atual['x'] << BitVecVal(1,n), prox['y'] == atual['y'] >> BitVecVal(1,n), prox['z'] == atual['z'], Or(And(ULT(atual['x'], BitVecVal(2**(n-1),n)),prox['c'] == 1),And(UGE(atual['x'], BitVecVal(2**(n-1),n)),prox['c'] == 4)))

    t2_1 = And(atual['c'] == 2, prox['x'] == atual['x'], prox['y'] == atual['y'] - 1, prox['z'] == atual['z'] + atual['x'], Or(And(ULT(prox['z'], BitVecVal(2**(n-1),n)),prox['c'] == 1),And(UGE(prox['z'], BitVecVal(2**(n-1),n)),prox['c'] == 4)))

    stop_transition = And(prox['c'] == atual['c'], valores_constantes, Or(And(atual['c'] == 4, prox['c'] == 4), And(atual['c'] == 5, prox['c'] == 5)))


    return Or(t0_1,t1_2,t1_3,t3_1,t2_1,stop_transition)

b. Usando $k$-indução verifique nesse SFOTS se a propriedade $$\;(x*y + z = a*b)\;$$ $$\;$$é um invariante do seu comportamento.

In [ ]:
def gera_auto(k,n,a,b):

    solver = Solver()

    estados = [states(i,n) for i in range(k)]

    solver.add(init(estados[0],a,b))

    for i in range(k-1):
        solver.add(trans(estados[i],estados[i+1],n))
    
    check = solver.check()

    if check == sat:
        m = solver.model()
        for i in range(k):
            print("Passo ",i)
            for v in estados[i]:
                print(v,"=", m[estados[i][v]])
    print(check)

gera_auto(20,8,120,2)


Passo  0
c = 0
x = 150
y = 2
z = 0
Passo  1
c = 1
x = 150
y = 2
z = 0
Passo  2
c = 3
x = 150
y = 2
z = 0
Passo  3
c = 4
x = 44
y = 1
z = 0
Passo  4
c = 4
x = 44
y = 1
z = 0
Passo  5
c = 4
x = 44
y = 1
z = 0
Passo  6
c = 4
x = 44
y = 1
z = 0
Passo  7
c = 4
x = 44
y = 1
z = 0
Passo  8
c = 4
x = 44
y = 1
z = 0
Passo  9
c = 4
x = 44
y = 1
z = 0
Passo  10
c = 4
x = 44
y = 1
z = 0
Passo  11
c = 4
x = 44
y = 1
z = 0
Passo  12
c = 4
x = 44
y = 1
z = 0
Passo  13
c = 4
x = 44
y = 1
z = 0
Passo  14
c = 4
x = 44
y = 1
z = 0
Passo  15
c = 4
x = 44
y = 1
z = 0
Passo  16
c = 4
x = 44
y = 1
z = 0
Passo  17
c = 4
x = 44
y = 1
z = 0
Passo  18
c = 4
x = 44
y = 1
z = 0
Passo  19
c = 4
x = 44
y = 1
z = 0
sat


c. Usando $k$-indução no FOTS acima e adicionando ao estado inicial  a condição  $$\,$$ $$\,(a < 2^{n/2})\land(b < 2^{n/2})\,$$ $$\,$$ verifique a segurança do programa; nomeadamente  prove que, com tal estado inicial, o estado de erro nunca é acessível.  